In [1]:

import pyarrow
from sklearn.pipeline import Pipeline
from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import *
from pymo.writers import *

from scipy.spatial.transform import Rotation as R

import joblib as jl



target_joints = ['Spine', 'Spine1', 'Spine2', 'Spine3', 'Neck', 'Neck1', 'Head',
                 'RightShoulder', 'RightArm', 'RightForeArm', 'RightHand',
                 'LeftShoulder', 'LeftArm', 'LeftForeArm', 'LeftHand']

In [2]:
def process_bvh(gesture_filename):
    p = BVHParser()

    data_all = list()
    data_all.append(p.parse(gesture_filename))

    data_pipe = Pipeline([
        ('dwnsampl', DownSampler(tgt_fps=20, keep_all=False)),
        ('root', RootTransformer('hip_centric')),
        ('mir', Mirror(axis='X', append=True)),
        ('jtsel', JointSelector(target_joints, include_root=True)),
        ('cnst', ConstantsRemover()),
        ('np', Numpyfier())
    ])

    out_data = data_pipe.fit_transform(data_all)
    jl.dump(data_pipe, os.path.join('../resource', 'data_pipe.sav'))

    # euler -> rotation matrix
    out_data = out_data.reshape((out_data.shape[0], out_data.shape[1], -1, 3))
    out_matrix = np.zeros((out_data.shape[0], out_data.shape[1], out_data.shape[2], 9))
    for i in range(out_data.shape[0]):  # mirror
        for j in range(out_data.shape[1]):  # frames
            r = R.from_euler('ZXY', out_data[i, j], degrees=True)
            out_matrix[i, j] = r.as_matrix().reshape(out_data.shape[2], 9)
    out_matrix = out_matrix.reshape((out_data.shape[0], out_data.shape[1], -1))

    return out_matrix[0], out_matrix[1]

In [4]:
bvh_file = "../../data/Test_data/Motion/TestSeq001.bvh"
loaded = process_bvh(bvh_file)
print("Finished!")

downsampling with rate: 3
RootTransformer
Mirror: X
JointSelector
Numpyfier


M:\Payam_Projects\Gesture2Vec\Gesture2Vec\scripts\pymo\preprocessing.py:315: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['%s_Yrotation'%joint] = pd.Series(data=signs[1]*track.values['%s_Yrotation'%joint], index=new_df.index)
M:\Payam_Projects\Gesture2Vec\Gesture2Vec\scripts\pymo\preprocessing.py:316: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df['%s_Zrotation'%joint] = pd.Series(data=signs[2]*track.values['%s_Zrotation'%joint], index=new_df.index)
M:\Payam_Projects\Gesture2Vec\Gesture2Vec\scripts\pymo\preprocessi

Finished!


In [6]:
def make_bvh_Trinity(save_path: str, filename_prefix: str, poses: np.ndarray) -> None:
    """Save Trinity input gesture data into a bvh file.

    Trinity data contains 135 dimensions of gestures.

    This function requires a saved Pipeline file located in:
    '../resource/data_pipe.sav'.

    Args:
        save_path: A string directory to save the
        filename_prefix: A string filename to use for the saved file.
        poses: An array of gestures data.
    """
    writer = BVHWriter()
    pipeline: Pipeline = jl.load("../resource/data_pipe.sav")

    # smoothing
    n_poses = poses.shape[0]
    out_poses = np.zeros((n_poses, poses.shape[1]))

    if n_poses > 15 and False:
        for i in range(poses.shape[1]):
            out_poses[:, i] = savgol_filter(
                poses[:, i], 15, 2
            )  # NOTE: smoothing on rotation matrices is not optimal
    else:
        out_poses = poses

    # rotation matrix to euler angles
    out_poses = out_poses.reshape(
        (out_poses.shape[0], -1, 9)
    )  # (n_frames, n_joints, 9)
    out_poses = out_poses.reshape((out_poses.shape[0], out_poses.shape[1], 3, 3))
    out_euler = np.zeros((out_poses.shape[0], out_poses.shape[1] * 3))
    for i in range(out_poses.shape[0]):  # frames
        r = R.from_matrix(out_poses[i])
        out_euler[i] = r.as_euler('ZXY', degrees=True).flatten()

    bvh_data = pipeline.inverse_transform([out_euler])

    out_bvh_path = os.path.join(save_path, filename_prefix + ".bvh")
    print(out_bvh_path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    with open(out_bvh_path, "w") as f:
        writer.write(bvh_data[0], f)

In [9]:
make_bvh_Trinity('output', 'pip_test', loaded[1])
print('Finished!')

M:\Payam_Projects\Gesture2Vec\Gesture2Vec\scripts\pymo\preprocessing.py:378: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  t2.values[d] = self.not_selected_values[d]
M:\Payam_Projects\Gesture2Vec\Gesture2Vec\scripts\pymo\preprocessing.py:378: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  t2.values[d] = self.not_selected_values[d]
M:\Payam_Projects\Gesture2Vec\Gesture2Vec\scripts\pymo\preprocessing.py:378: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

output\pip_test.bvh
Finished!
